In [25]:
import torch
from torch import nn,optim
from torch.utils.data import Dataset,DataLoader

In [26]:
import torch
from torch import nn,optim
from torch.utils.data import Dataset,DataLoader

class SelfAttention(nn.Module):
  def __init__(self,embed_size,heads):
    super(SelfAttention,self).__init__()
    self.embed_size=embed_size
    self.heads=heads
    self.head_dim=embed_size//heads
    assert(self.head_dim*heads==embed_size),"Embed size should be divisible by heads"
    self.values=nn.Linear(self.head_dim,self.head_dim,bias=False)
    self.keys=nn.Linear(self.head_dim,self.head_dim,bias=False)
    self.queries=nn.Linear(self.head_dim,self.head_dim,bias=False)
    self.fc_out=nn.Linear(heads*self.head_dim,embed_size)
    self.softmax = nn.Softmax(dim=3)

  def forward(self,values,keys,query,mask):
    N=query.shape[0]
    value_len,key_len,query_len=values.shape[1],keys.shape[1],query.shape[1]
    #splitting the embeddings into head pieces
    values=values.reshape(N,value_len,self.heads,self.head_dim)
    keys=keys.reshape(N,key_len,self.heads,self.head_dim)
    queries=query.reshape(N,query_len,self.heads,self.head_dim)
    values=self.values(values)
    keys=self.keys(keys)
    queries=self.queries(queries)
    energy=torch.einsum("nqhd,nkhd->nhqk",[queries,keys])

    if mask is not None:
      energy=energy.masked_fill(mask==0,float("-1e20"))

    attention=self.softmax(energy/(self.embed_size**(1/2)))
    out=torch.einsum("nhql,nlhd->nqhd",[attention,values]).reshape(N,query_len,self.heads*self.head_dim)
    out=self.fc_out(out)
    return out

In [27]:
class Transformer(nn.Module):
  def __init__(self,embed_size,heads,dropout,forward_expansion):
    super(Transformer,self).__init__()
    self.attention=SelfAttention(embed_size,heads)
    self.norm1=nn.LayerNorm(embed_size)
    self.norm2=nn.LayerNorm(embed_size)

    self.feed_forward=nn.Sequential(
        nn.Linear(embed_size,forward_expansion*embed_size),
        nn.ReLU(),
        nn.Linear(forward_expansion*embed_size,embed_size)

    )
    self.dropout=nn.Dropout(dropout)

  def forward(self,value,key,query,mask):
    attention=self.attention(value,key,query,mask)

    x=self.dropout(self.norm1(attention+query))
    forward=self.feed_forward(x)
    out=self.dropout(self.norm2(forward+x))
    return out

In [28]:
class Encoder(nn.Module):
  def __init__(
    self,
    src_vocab_size,
    embed_size,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    max_length):
    super(Encoder,self).__init__()
    self.embed_size=embed_size
    self.device=device
    self.word_embedding=nn.Embedding(src_vocab_size,embed_size)
    self.position_embedding=nn.Embedding(max_length,embed_size)
    self.layers=nn.ModuleList([
        Transformer(
            embed_size,
            heads,
            dropout=dropout,
            forward_expansion=forward_expansion
        )
        for _ in range(num_layers)
    ])
    self.dropout=nn.Dropout(dropout)
  def forward(self,x,mask):
    N,seq_length=x.shape
    positions=torch.arange(0,seq_length).expand(N,seq_length).to(self.device)
    out=self.dropout(self.word_embedding(x)+self.position_embedding(positions))
    for layer in self.layers:
      out=layer(out,out,out,mask)
    return out

In [29]:
class DecoderBlock(nn.Module):
  def __init__(self,embed_size,heads,forward_expansion,dropout,device):
    super(DecoderBlock,self).__init__()
    self.attention=SelfAttention(embed_size,heads)
    self.norm=nn.LayerNorm(embed_size)
    self.transformer_block=Transformer(
        embed_size,heads,dropout,forward_expansion)
    self.dropout=nn.Dropout(dropout)

  def forward(self,x,value,key,src_mask,trg_mask):
    attention=self.attention(x,x,x,trg_mask)
    query=self.dropout(self.norm(attention+x))
    out=self.transformer_block(value,key,query,src_mask)
    return out

In [30]:
class Decoder(nn.Module):
  def __init__(self,
               trg_vocab_size,
               embed_size,
               num_layers,
               heads,
               forward_expansion,
               dropout,
               device,
               max_length):
    super(Decoder,self).__init__()
    self.device=device
    self.word_embedding=nn.Embedding(trg_vocab_size,embed_size)
    self.position_embedding=nn.Embedding(max_length,embed_size)
    self.layers=nn.ModuleList(
        [DecoderBlock(embed_size,heads,forward_expansion,dropout,device)
        for _ in range(num_layers)]
    )
    self.fc_out=nn.Linear(embed_size,trg_vocab_size)
    self.dropout=nn.Dropout(dropout)

  def forward(self,x,enc_out,src_mask,trg_mask):
    N,seq_length=x.shape
    positions=torch.arange(0,seq_length).expand(N,seq_length).to(self.device)
    x=self.dropout((self.word_embedding(x)+self.position_embedding(positions)))

    for layer in self.layers:
      x=layer(x,enc_out,enc_out,src_mask,trg_mask)

    out=self.fc_out(x)
    return out


In [31]:
class TransformerModel(nn.Module):
    def __init__(self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=256,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cuda",
        max_length=100):

        super(TransformerModel, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )
        self.decoder=Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )
        self.src_pad_idx=src_pad_idx
        self.trg_pad_idx=trg_pad_idx
        self.device=device

    def make_src_mask(self,src):
      src_mask=(src!=self.src_pad_idx).unsqueeze(1).unsqueeze(2)
      return src_mask.to(self.device)

    def make_trg_mask(self,trg):
      N,trg_len=trg.shape
      trg_mask=torch.tril(torch.ones((trg_len,trg_len))).expand(
          N,1,trg_len,trg_len
      )
      return trg_mask.to(self.device)

    def forward(self,src,trg):
      src_mask=self.make_src_mask(src)
      trg_mask=self.make_trg_mask(trg)
      enc_src=self.encoder(src,src_mask)
      out=self.decoder(trg,enc_src,src_mask,trg_mask)
      return out



In [32]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [33]:

class TextDataset(Dataset):
    def __init__(self, src_data, trg_data, src_tokenizer, trg_tokenizer, max_length):
        self.src_data = src_data
        self.trg_data = trg_data
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer
        self.max_length = max_length


    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src_text = self.src_data[idx]
        trg_text = self.trg_data[idx]

        src_tokens = [self.src_tokenizer.sos_token] + self.src_tokenizer.encode(src_text) + [self.src_tokenizer.eos_token]
        trg_tokens = [self.trg_tokenizer.sos_token] + self.trg_tokenizer.encode(trg_text) + [self.trg_tokenizer.eos_token]


        src_padded = src_tokens + [self.src_tokenizer.pad_token] * (self.max_length - len(src_tokens))
        trg_padded = trg_tokens + [self.trg_tokenizer.pad_token] * (self.max_length - len(trg_tokens))


        src_padded = src_padded[:self.max_length]
        trg_padded = trg_padded[:self.max_length]


        src_ids = self.src_tokenizer.convert_tokens_to_ids(src_padded)
        trg_ids = self.trg_tokenizer.convert_tokens_to_ids(trg_padded)

        return torch.tensor(src_ids), torch.tensor(trg_ids)

class SimpleTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.token_to_id = {token: i for i, token in enumerate(vocab)}
        self.id_to_token = {i: token for i, token in enumerate(vocab)}
        self.pad_token = "<PAD>"
        self.sos_token = "<SOS>"
        self.eos_token = "<EOS>"
        self.unk_token = "<UNK>"
        self.pad_token_id = self.token_to_id.get(self.pad_token)
        self.sos_token_id = self.token_to_id.get(self.sos_token)
        self.eos_token_id = self.token_to_id.get(self.eos_token)
        self.unk_token_id = self.token_to_id.get(self.unk_token)

    def encode(self, text):
        return [self.token_to_id.get(word, self.unk_token_id) for word in text.split()]

    def decode(self, ids):
        return " ".join([self.id_to_token.get(id) for id in ids])

    def convert_tokens_to_ids(self, tokens):
        return [self.token_to_id.get(token, self.unk_token_id) for token in tokens]



In [ ]:

# eg
src_sentences = ["hello world", "how are you", "i am fine"]
trg_sentences = ["bonjour le monde", "comment vas tu", "je vais bien"]


all_words = []
for s in src_sentences + trg_sentences:
    all_words.extend(s.split())
vocab_set = set(all_words)
vocab_list = list(vocab_set) + ["<PAD>", "<SOS>", "<EOS>", "<UNK>"]

src_tokenizer = SimpleTokenizer(vocab_list)
trg_tokenizer = SimpleTokenizer(vocab_list)

SRC_VOCAB_SIZE = len(src_tokenizer.vocab)
TRG_VOCAB_SIZE = len(trg_tokenizer.vocab)
MAX_LENGTH = 10
SRC_PAD_IDX = src_tokenizer.pad_token_id
TRG_PAD_IDX = trg_tokenizer.pad_token_id

train_dataset = TextDataset(src_sentences, trg_sentences, src_tokenizer, trg_tokenizer, MAX_LENGTH)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)



embed_size = 256
num_layers = 3
heads = 8
forward_expansion = 4
dropout = 0.1
learning_rate = 3e-4
num_epochs = 100


model = TransformerModel(
    src_vocab_size=SRC_VOCAB_SIZE,
    trg_vocab_size=TRG_VOCAB_SIZE,
    src_pad_idx=SRC_PAD_IDX,
    trg_pad_idx=TRG_PAD_IDX,
    embed_size=embed_size,
    num_layers=num_layers,
    forward_expansion=forward_expansion,
    heads=heads,
    dropout=dropout,
    device=device,
    max_length=MAX_LENGTH
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [34]:



print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch_idx, (src, trg) in enumerate(train_dataloader):
        src = src.to(device)
        trg = trg.to(device)

        output = model(src, trg[:, :-1])

        output = output.reshape(-1, output.shape[2])
        target = trg[:, 1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader):.4f}")


Starting training...
Epoch [1/100], Loss: 2.5348
Epoch [2/100], Loss: 0.7282
Epoch [3/100], Loss: 0.4859
Epoch [4/100], Loss: 0.4139
Epoch [5/100], Loss: 0.2752
Epoch [6/100], Loss: 0.1995
Epoch [7/100], Loss: 0.0867
Epoch [8/100], Loss: 0.0450
Epoch [9/100], Loss: 0.0193
Epoch [10/100], Loss: 0.0166
Epoch [11/100], Loss: 0.0085
Epoch [12/100], Loss: 0.0069
Epoch [13/100], Loss: 0.0063
Epoch [14/100], Loss: 0.0046
Epoch [15/100], Loss: 0.0041
Epoch [16/100], Loss: 0.0035
Epoch [17/100], Loss: 0.0028
Epoch [18/100], Loss: 0.0030
Epoch [19/100], Loss: 0.0023
Epoch [20/100], Loss: 0.0017
Epoch [21/100], Loss: 0.0017
Epoch [22/100], Loss: 0.0014
Epoch [23/100], Loss: 0.0017
Epoch [24/100], Loss: 0.0013
Epoch [25/100], Loss: 0.0013
Epoch [26/100], Loss: 0.0011
Epoch [27/100], Loss: 0.0012
Epoch [28/100], Loss: 0.0010
Epoch [29/100], Loss: 0.0009
Epoch [30/100], Loss: 0.0008
Epoch [31/100], Loss: 0.0010
Epoch [32/100], Loss: 0.0010
Epoch [33/100], Loss: 0.0008
Epoch [34/100], Loss: 0.0008
Ep